In [ ]:
#Define things, build torsion profile
t = Tableau([[1]])
d_start = 1
_mm_= [int(n) for n in raw_input("Enter supertorsion profile as a sequence of numbers separated by spaces:").split()]
k = len(_mm_)+1
_m_ = []
for n in range(2,len(_mm_)+2):
    i = 0
    while i < _mm_[n-2]:
        _m_.append(n)
        i+=1
    _m_.append(0)
_m_.extend([0]*(k-2))
print "_mm_ =", _mm_
print "_m_ = ", _m_
g = len(_m_)+1

#Calculate actual initial placement of each number 
def xi(l):
    if l < g-k+2:
        return l-1
    else:
        return l-k-1
    
#Calculate torsion of each loop, with an arbitrary large prime for loops that are undetermined
def T(l):
    if l>len(_m_)-k+1:
        return k
    elif _m_[l-2]>0:
        return _m_[l-2]
    elif _m_[l-2]==0:
        return 104729
    else:
        return k
    
#Iterate along diagonals to build the "stack" of tableaux corresponding to our chain of loops
while d_start < g-k+1:
    x = 0
    y = d_start
    while y> -1:
        if x==0:
            a = 0
            b = t.entry((y-1,0))
        elif y==0:
            a = t.entry((0,x-1))
            b = 0
        else:
            a = t.entry((y,x-1))
            b = t.entry((y-1,x))
        d = max(a,b)+1
        while ((y-x) - xi(d)) % T(d) != 0:
            if d>g:
                break
            else:
                d+=1
        t=t.add_entry((y,x), d)
        x+=1
        y-=1
    d_start += 1
t=t.add_entry((g-k,1), g)
t=t.restrict(g)


#Calculate rank of each multiple of D
diagonals = {c : [] for c in range(1,g-k)}
for c in diagonals.keys():
    for((y,x)) in t.cells():
        if y-x == g-c*k-1:
            diagonals[c].append(x)
rank = {c : [] for c in range(1,g-k)}
maxmult = 0
for c in rank.keys():
    if diagonals[c] != []:
        rank[c] = max(diagonals[c])
    if rank[c] != []:
        #print "The rank of ",c,"D is ",rank[c]
        maxmult += 1
        
#Cut to corners, so there aren't stray numbers that aren't part of the tableau for cD
corners = {c : [] for c in range(1,maxmult+1)}
corners[1].append(g-k)
corners[1].append(1)
for c in range(2,maxmult+1):
    y = [0]
    for n in range(1,g-k):
        if ((n,rank[c])) in t.cells():
            y.append(n)
    corners[c].append(max(y))        
    corners[c].append(rank[c])
for c in corners:
    for ((y,x)) in t.cells():
        if x> corners[maxmult][1] or y > corners[c][0] and x> corners[c-1][1]:
            t=t.add_entry((y,x),0)
        else:
            continue
t=t.anti_restrict(0)
t.pp()
        
#Calculate Scrollar Invariants
sis = {t : [] for t in range(1,k+1)}
#Start with aisum=1 so we don't have to subtract 1 in check every time
aisum = 1
j = 1
t = 1
while j < k+1:
    if sis[j] == []:
        if all(rank[v] >= j*t + j -t -aisum for v in range(t,maxmult+1)):
            sis[j] = t-1
            aisum += t-1
            j += 1
        else:
            t += 1
#print "aisum =", aisum
#print maxmult
print "The scrollar invariants are:",sis
#print "g+k-1-aisum = ", g+k-1-aisum
